In [1]:
!pip install -qU xpander-sdk llama-index llama-index-llms-openai python-dotenv openai


In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [3]:
llm = OpenAI(model="gpt-4o")
llama_agent = ReActAgent.from_llm(llm=llm, verbose=True)
llama_agent.chat("What is the weather in Tokyo?")

Added user message to memory: What is the weather in Tokyo?


AgentChatResponse(response="I'm unable to provide real-time weather updates. For the current weather in Tokyo, I recommend checking a reliable weather website or app.", sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [4]:
from openai import OpenAI
from xpander_sdk import XpanderClient
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

xpander_agent.invoke_agent("Summarize the profile of david twizer from xpander.ai")
openai_client = OpenAI(api_key=OPENAI_API_KEY)
xpander_agent.memory.initialize(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)
step = 1
while not xpander_agent.is_finished():
    print("-"*100)
    print(f"Step {step}")
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(),
                tool_choice="auto",
                temperature=0.0
        )
            
    xpander_agent.memory.add_messages(response.model_dump())
    print("model response", response.model_dump())
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    for tool_call in tool_calls:
        print("tool name: ", tool_call.name)
        print("tool call generated payload: ", tool_call.payload)
    tool_responses = xpander_agent.run_tools(tool_calls=tool_calls)
    for tool_response in tool_responses:
        print("tool name: ", tool_response.function_name)
        print("tool status code: ", tool_response.status_code)
        print("tool result: ", tool_response.result)
    step += 1
# result (re fetch execution result)
# IMPORTANT: LAST TOOL IS xpfinish-agent-execution-finished WHICH IS AGENT-END with PARSING and may be slower due to inference times
execution_result = xpander_agent.execution.fetch(agent=xpander_agent,execution_id=xpander_agent.execution.id)
print("status", execution_result.status)
print("result", execution_result.result)

----------------------------------------------------------------------------------------------------
Step 1
model response {'id': 'chatcmpl-AvClyhTq8PKq4yDbmPhdDtfRjjiUJ', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_NQlSLkWrmVfcRsol0zkBMXhy', 'function': {'arguments': '{"queryParams":{"firstName":"David","lastName":"Twizer","company":"Xpander.ai"}}', 'name': 'LinkedInProfileServiceSearchProfilesByCriteria'}, 'type': 'function'}]}}], 'created': 1738197158, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_4691090a87', 'usage': {'completion_tokens': 37, 'prompt_tokens': 1404, 'total_tokens': 1441, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_